In [2]:
import keras_nlp

In [3]:
#Fixed parameters
PROCESSING_FIXED = {'start_char': "<s>",
                    'end_char': "</s>", 
                    'pad_char': "<pad>",
                    'mas_char': "<mask>"}

INDICES_TOKEN = {0: 'c',
                 1: 'C',
                 2: '(',
                 3: ')',
                 4: 'O',
                 5: '1',
                 6: '2',
                 7: '=',
                 8: 'N',
                 9: '@',
                 10: '[',
                 11: ']',
                 12: 'n',
                 13: '3',
                 14: 'H',
                 15: 'F',
                 16: '4',
                 17: '-',
                 18: 'S',
                 19: 'Cl',
                 20: '/',
                 21: 's',
                 22: 'o',
                 23: '5',
                 24: '+',
                 25: '#',
                 26: '\\',
                 27: 'Br',
                 28: 'P',
                 29: '6',
                 30: 'I',
                 31: '7',
                 32: PROCESSING_FIXED['start_char'],
                 33: PROCESSING_FIXED['end_char'],
                 34: PROCESSING_FIXED['pad_char'],
                 35: PROCESSING_FIXED['mas_char']}                
TOKEN_INDICES = {v: k for k, v in INDICES_TOKEN.items()}


In [5]:
tokenizer = keras_nlp.models.RobertaTokenizer(vocabulary=TOKEN_INDICES, merges=['C l', 'B r'])

I0000 00:00:1723387710.773907   36131 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-11 15:48:30.925766: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
preprocessor = keras_nlp.models.RobertaMaskedLMPreprocessor(
    tokenizer,
    sequence_length=90,
    mask_selection_rate=0.15,
    mask_selection_length=16,
    mask_token_rate=0.8,
    random_token_rate=0
)

In [7]:
back = keras_nlp.models.RobertaBackbone(
    vocabulary_size=36,
    num_layers=4,
    num_heads=4,
    hidden_dim=256,
    intermediate_dim=512,
    max_sequence_length=90,
    dropout_rate=0.1,
)

In [8]:
model = keras_nlp.models.RobertaMaskedLM(back, preprocessor)

In [9]:
model.summary()

Preprocessor: "roberta_masked_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ roberta_tokenizer_1 (RobertaTokenizer)             │                                                  36 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "roberta_masked_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ roberta_backbone              │ (None, None, 256)         │       2,141,184 │ padding_mask[0][0],        │
│ (RobertaBackbone)             │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ mask_positions (InputLayer)   │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ mlm_head (MaskedLMHead)       │ (None, None, 36)          │          75,556 │ roberta_backbone[0][0],    │
│                               │                           │                 │ mask_positions[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,207,524 (8.42 MB)

 Trainable params: 2,207,524 (8.42 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit()

In [ ]:
#Create a class for the model using what we have above. It is important to put the model in a hypermodel class in order to be able to tune hyperparameters with keras-tuner. 
#Follow this tutorial until the Tune model training part: https://keras.io/guides/keras_tuner/getting_started/#tune-model-training
#The parameters to tune are the following:
#hidden dimension, number of attention heads, dropout, intermediate dimention, number of hidden layers, and the learning rate.
#The ranges to use are:
# hidden_dim: 128 to 512
# num_heads: 2, to 8
# dropout: 0.1 to 0.3
# intermediate_dim: 256 to 1024
# num_layers: 2 to 6
# learning_rate: 1e-5 to 5e-5